# Generating Templates

Assuming that the parent directory of the ubbr root directory is in the python path try 

In [1]:
from ubbr.engine.core import Ubbr

In [2]:
u = Ubbr('Hello world!')

In [3]:
u.get_template()

'Hello world!'

Anything between {% ubbr %} and {% endubbr %} tags is treated as executable Python code 

In [4]:
v = Ubbr('Hello {% ubbr %} a = 1 \nprint(a){% endubbr %}')

The python code must, of course, be properly indented. Try inserting a space after \n in the above source string

In [5]:
v.get_template()

'Hello {{ ubbrvalues.0 }}'

Observe that in the template output the {% ubbr %} ... {% endubbr %} section has been replaced by {{ ubbervalues.0 }} which a conventional syntax for html template variable interpolation. Thus we can use the output of Ubbr.template as a template in web application frameowrks like Django/Flask. Where do we get the ubbervalues variable?

In [6]:
v.get_context()

1


([''], [])

The first entry in this pair is the value of ubbrvalues. This is not a very interesting example

In [7]:
w = Ubbr('Hello {% ubbr %} echo(" world!"){% endubbr%}')
w.get_template()

'Hello {{ ubbrvalues.0 }}'

In [8]:
w.get_context()

([' world!'], [])

So the basic useage pattern in a web app is ...

Create an Ubbr instance and get the source from somewhere e.g. database

```
u = Ubbr([source])
```

generate a template with 
```
template = u.get_template()
```

generate the value of the ubbrvalues context variable
```
ubbrvalues = u.get_context()[0]
```

render the template and serve using your frameowrk of choice. Have a look in the ubbr_webapp folder to see a Django based example

## Options for ```Ubbr```

### XML style tags
You can specify xml style tags to mark the embedded code as follows

In [9]:
ub = Ubbr('Hello <ubbr>echo("world!")</ubbr>',tag_style='xml')

In [10]:
ub.get_context()

(['world!'], [])

# Inputs

in the engine/inputs.py there are classes for handling end user inputs. All of these classes inherit from 
```ubbr.engine.inputs.BaseInput``` class. They serve two basic purposes

1. When echoed into the html they produce an appropriate html input element

2. When echoed into the html they add an item to the ```ubbr_state.data``` object that is used to pass data to and from the problem code. This object is a list that is returned as Ubbr.get_context()[1]

In [11]:
u = Ubbr('{% ubbr %}echo(StringInput("a")) {% endubbr %} and another {% ubbr %} echo(IntegerInput(10)) {% endubbr %}')

In [12]:
cntxt = u.get_context()

In [13]:
cntxt[0]

["<div class='ubbr-input' id='ubbr-input-0'><input type='text' name='ubbr-input-0'></input></div>",
 "<div class='ubbr-input' id='ubbr-input-1'><input type='text' name='ubbr-input-1'></input></div>"]

In [14]:
cntxt[1]

[{'answer': 'a', 'data_id': 0, 'data_type': 'StringInput'},
 {'answer': 10, 'data_id': 1, 'data_type': 'IntegerInput'}]

Observe that the input elements in the list ```cntxt[0]``` have id attributes and name attributes that include numbers equal to the ```data_id``` attribute in the corresponding dictionary in ```cntxt[1]```. Thus our web application can store the value of ```cntxt[1]``` somewhere and use it to grade any user input.

## Graders
The final core piece of system are the grader classes. For each subclass of '''BaseInput''' there is a corresponding subclass of ```BaseGrader``` that takes the output of ```BaseInput.data()``` together with a
guess and returns a dictionary as below

In [15]:
from ubbr.engine.inputs import StringInput

In [16]:
inp = StringInput('the correct string')

In [17]:
from ubbr.engine.graders.graders import StringInputGrader

In [18]:
grader = StringInputGrader()

now we can use the grade method of ```grader``` to grade user input

In [19]:
data = inp.data()

In [20]:
grader.grade('incorrect answer',data)

{'data': {}, 'max_score': 1, 'score': 0}

In [21]:
grader.grade('the correct string',data)

{'data': {}, 'max_score': 1, 'score': 1}

observe that the ```grade``` method returns a ```dict``` that has keys ```score``` and ```max_score``` whose values  together quantify the correctness of the string that is passed as the first argument

## Input types

So far we we have implemented the following subclasses of ```BaseInput```.

### ```StringInput```

### ```IntegerInput```

### ```DecimalInput```